In [1]:
import numpy as np
import pandas as pd
import datetime
import pytz
import requests
import wordle_cheater as wrdlc
from IPython import display

___
### Basic usage of the ```wordle_cheater```

Create an instance of the class ```wordl``` in the```wordle_cheater``` module to help solve for the wordle solution **whine**.

In [2]:
# instantiate the wordl class
wdl = wrdlc.wordl()
# get todays word
td_word = wdl.todays_word

___
### Add the word *alert* to the ```wdl``` instance.

   * Each word that you add to the ```wordl``` instance requires an array of 5 integers between 1 and 3 (inclusive), for each off the 5 letters of the word that you add. 
   * The array of 5 integers between 1 and 3 tells the ```wordl.try_it()``` method one of 3 things about the respective letter:
     1. if the letter is in the solution word, and the letter's position in the word is the same as the letter's position in the solution word, then the letter code is **1** for that letter;
     2. if the letter is in the solution word, but NOT in the same position, then the letter code is **2**;
     3. if the letter is NOT at all in the solution word, then the letter code is **3**.

In the word **alert**:
  1. The letter **a** is not in the word **whine**, so it's letter code = 3
  2. The letter **l** is not in the word **whine**, so it's letter code = 3
  3. The letter **e** is in the word **whine**, but NOT is the correct position, so it's letter code = 2
  4. The letter **r** is not in the word **whine**, so it's letter code = 3
  4. The letter **t** is not in the word **whine**, so it's letter code = 3

So, for the word **alert**, the letter_codes array is:
  * ```[3,3,2,3,3]```

In [3]:
# add the word 'alert' to the instance
wdl.add_word('alert',[3,3,2,3,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
df

,word,count,probability
1,video,365410017,0.10670
2,phone,256643812,0.07494
3,index,242826246,0.07091
4,being,242783091,0.07090
5,women,242520455,0.07082
...,...,...,...
140,chide,66368,0.00002
141,bused,62772,0.00002
142,eying,31044,0.00001
143,eking,26695,0.00001


___
### Add the word *noisy* to the ```wdl``` instance.

In the word **noisy**:
  1. The letter **n** is in the word **whine**, but NOT is the correct position, so it's letter code = 2
  2. The letter **o** is not in the word **whine**, so it's letter code = 3
  3. The letter **i** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **s** is not in the word **whine**, so it's letter code = 3
  4. The letter **t** is not in the word **whine**, so it's letter code = 3

So, for the word **noisy**, the letter_codes array is:
  * ```[2,3,1,3,3]```

In [4]:
wdl.add_word('noisy',[2,3,1,3,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
df

,word,count,probability
1,being,242783091,0.95152
2,knife,11297715,0.04428
3,whine,832743,0.00326
4,feign,116753,0.00046
5,deign,94806,0.00037
6,eking,26695,0.00010


### Since *being* is the most probable word, add it

In the word **being**:
  1. The letter **b** is not in the word **whine**, so it's letter code = 3
  2. The letter **e**  is in the word **whine**, BUT is is NOT in the correct position, so it's letter code = 2
  3. The letter **i** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **n** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **g** is not in the word **whine**, so it's letter code = 3

So, for the word **being**, the letter_codes array is:
  * ```[3,2,1,1,3]```

In [5]:
wdl.add_word('being',[3,2,1,1,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
df

,word,count,probability
1,whine,832743,1.0


___
### The only word left is *whine*.
___

___
### Methods to create the letter lists automatically

In [6]:
def get_letter_list(your_word,the_word):
    nums =[]
    for i,l in enumerate(list(your_word)):
        if l == the_word[i]:
            nums.append(1)
        elif l in the_word:
            nums.append(2)
        else:
            nums.append(3)
    return nums    
        
def letter_list_to_str(letter_list):
    return ''.join(
        [str(v) for v in letter_list]
    )
def get_filter(your_word,the_word):
    letter_string = letter_list_to_str(get_letter_list(your_word,td_word))
    return your_word + "," + letter_string

### Below are methods that will find the solution to the the word, given the date, and your 2 first words.


In [22]:
wdl2 = wrdlc.wordl()

solution = wdl2.df_word_history.iloc[7].solution.lower()
for w in ['alert','noisy']:
    wdl2.add_word(w,get_letter_list(w,solution))
df = wrdlc.filter_words(wdl2.try_it()) 
df.probability = df.probability.round(5)
df
    

,word,count,probability
1,excel,14133468,1.0


'excel'

In [9]:
# define your first 2 words
initial_words = ['alert','noisy']
# create initial filter string that gets used in the url for the 
#   http call to the wordl server at http://127.0.0.1:8000/wordl
initial_filters = ','.join(
    np.array(
        [
            get_filter(word,get_filter(word,td_word))
            for word in initial_words
        ]
    ).reshape(-1).tolist()
)

# The loop below will populate the string called filters.  
#   This string will be added to during the for loop below.
filters = initial_filters
# results_list holds DataFrames that show possible words after 
#   words are added to the filters string, and the server is called again
results_list = []
for _ in range(3):
    url = f'http://127.0.0.1:8000/wordl?filters={filters}'
    response = requests.get(url)
    resp_dict = response.json()    
    df_results = pd.DataFrame(resp_dict)
    results_list.append(df_results.copy())
    if len(df_results)<=1:
        break
    next_word = df_results.iloc[0].word
    new_filter = get_filter(next_word,td_word)
    filters = filters + "," + new_filter
    

### Show resulting DataFrames

In [10]:
for df_rl in results_list:
    display.display(df_rl)

,word,count,probability
0,shirt,40501964,0.58533
1,strip,18711440,0.27042
2,skirt,9981485,0.14425


,word,count,probability
0,skirt,9981485,1.0
